In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
import cartopy as ccrs
from matplotlib import pyplot as plt
from netCDF4 import Dataset
%matplotlib inline

In [ ]:
precipforcasturl = "http://iridl.ldeo.columbia.edu/SOURCES/.Models/.SubX/.ESRL/.FIMr1p1/.hindcast/.pr/L/2.5/31.5/RANGEEDGES/%5BL%5D/average/X/-130.25/-99.75/RANGE/Y/24.75/50.25/RANGE/dods "
rainforcast=xr.open_dataset(precipforcasturl, chunks={'S': 100})
sstforecasturl="http://iridl.ldeo.columbia.edu/SOURCES/.Models/.SubX/.ESRL/.FIMr1p1/.hindcast/.ts/L/2.5/31.5/RANGEEDGES/%5BL%5Daverage/Y/-5/5/RANGEEDGES/X/190/240/RANGEEDGES/dods "
sstforcast=xr.open_dataset(sstforecasturl, chunks={'S': 100})
windforcasturl="http://iridl.ldeo.columbia.edu/SOURCES/.Models/.SubX/.ESRL/.FIMr1p1/.hindcast/.ua/L/2.5/31.5/RANGEEDGES/%5BL%5D/average/X/220/240/RANGE/Y/27/40/RANGE/[X/Y]average/dods"
windforcast=xr.open_dataset(windforcasturl,chunks={"S":100})

In [ ]:
windforcast=windforcast.sel(P=200)
sstforcast=sstforcast.mean(["X","Y"])


In [ ]:
windforcast


In [ ]:
sstforcast

In [ ]:
rainforcast


Make Adjusted time arrays: 

In [ ]:
td = np.timedelta64(15,'D')

In [ ]:
pr=np.array(rainforcast.pr)
T=np.array(rainforcast.coords["S"])+td
M=np.array(rainforcast.coords["M"])
pforcast_adjtime= xr.Dataset(data_vars={'pr':    (('T', 'M','Y','X'), pr)},coords={'T': T,'M': M,'Y':rainforcast.coords["Y"],'X':rainforcast.coords["X"]},)
pforcast_adjtime

In [ ]:
ua=np.array(windforcast.ua)
T=np.array(windforcast.coords["S"])+td
M=np.array(windforcast.coords["M"])
np.shape(ua)

In [ ]:
wforcast_adjtime= xr.Dataset(data_vars={'ua':  (('T', 'M'), ua)},coords={'T': T,'M': M},)
wforcast_adjtime

In [ ]:
ts=np.array(sstforcast.ts)
T=np.array(windforcast.coords["S"])+td
M=np.array(windforcast.coords["M"])
sstforcast_adjtime= xr.Dataset(data_vars={'ts':  (('T', 'M'), ts)},coords={'T': T,'M': M},)
sstforcast_adjtime

In [ ]:
import cartopy.feature as cfeature
def plotmap(data,title,resolution="one",ax=plt.axes(projection=ccrs.crs.Robinson()),colorbar="auto"):
    central_lon, central_lat = -115, 40
    extent = [-130,-100,25, 50]
    ax.set_title(title)
    ax.set_extent(extent)
    ax.gridlines()
    ax.coastlines(resolution='50m')
    ax.add_feature(cfeature.STATES)
    lons, lats = getlonlat(resolution)
    cb=ax.contourf(lons, lats, data,transform=ccrs.crs.PlateCarree(),cmap='nipy_spectral')
    if colorbar=="auto":
        plt.colorbar(cb, cmap='nipy_spectral', orientation='vertical',ticklocation='auto')
    else:
        return cb
def getlonlat(resolution):
    if resolution=="half":
        lats = np.linspace(24.75,50.25, 52)
        lons= np.linspace(-130.25,-99.75,62)
    else:
        lats = np.linspace(24,51, 28)
        lons= np.linspace(-131,-99,33)
    return lons, lats

In [ ]:
month=["Nov","Dec","Jan","Feb"]
monthnum=[10,11,1,2]
fig, (ax1, ax2,ax3,ax4) = plt.subplots(ncols=4, subplot_kw={'projection': ccrs.crs.Robinson()},figsize=(30,5))
for i,j in enumerate([ax1,ax2,ax3,ax4]):
    pmonth=pforcast_adjtime.sel(T=pforcast_adjtime["T.month"]==monthnum[i])
    rainanom=pmonth - pmonth.mean('T')
    sstmonth=sstforcast_adjtime.sel(T=wforcast_adjtime["T.month"]==monthnum[i])
    data1=rainanom
    data2=sstmonth
    cov=(data2.ts*data1.pr).mean(["T","M"])
    s1=data1.pr.std(["T","M"])
    s2=data2.ts.std(["T","M"])
    plot=cov/s1/s2
    cb=plotmap(plot,"Correlation Forecast Precip and SST "+month[i],resolution="one",ax=j,colorbar="nonauto")
cbar=fig.add_axes([.9, 0.15, 0.02, 0.5])
fig.colorbar(cb,cax=cbar)


In [ ]:
month=["Nov","Dec","Jan","Feb"]
monthnum=[10,11,1,2]
fig, (ax1, ax2,ax3,ax4) = plt.subplots(ncols=4, subplot_kw={'projection': ccrs.crs.Robinson()},figsize=(30,5))
for i,j in enumerate([ax1,ax2,ax3,ax4]):
    pmonth=pforcast_adjtime.sel(T=pforcast_adjtime["T.month"]==monthnum[i])
    rainanom=pmonth - pmonth.mean('T')
    wmonth=wforcast_adjtime.sel(T=wforcast_adjtime["T.month"]==monthnum[i])
    windanom=wmonth-wmonth.mean("T")
    data1=rainanom
    data2=windanom
    cov=(data2.ua*data1.pr).mean(["T","M"])
    s1=data1.pr.std(["T","M"])
    s2=data2.ua.std(["T","M"])
    plot=cov/s1/s2
    cb=plotmap(plot,"Correlation Forecast Precip and Wind "+month[i],resolution="one",ax=j,colorbar="nonauto")
cbar=fig.add_axes([.9, 0.15, 0.02, 0.5])
fig.colorbar(cb,cax=cbar)

In [ ]:
#A linear algebra regression function 
def regress(a,b):
    A=np.matmul(np.transpose(a),a)
    B=np.matmul(np.transpose(a),b)
    return np.linalg.solve(A,B)
    

    
#tshape is a multiplication of other variables (4 M variables* t, the resulting matrices will be 4t long)
#dependent is the b matrix 
#indenepent is one of the a matrices
#precipiation is our constant a matrix 
# this function computes two coefficients for ax=b and correlates their respectives squared errors 


def partialcorr(dependent,independent,precip,month,shape,tshape):
    error1=np.empty((shape[0],shape[1],tshape*4))
    b2=dependent
    a=np.ones((tshape*4,2))
    for k,m in enumerate([1.0,2.0,3.0,4.0]):
            a[k*tshape:(k+1)*tshape,0]=np.array(independent.sel(M=m))
    if shape==(52,62):
        x,y=getlonlat("half")
    else:
         x,y=getlonlat("one")
    for i in range(shape[1]):
        for j in range(shape[0]):
            b_array1=np.ones((tshape*4))
            for k,m in enumerate([1.0,2.0,3.0,4.0]):
                    b_array1[k*tshape:(k+1)*tshape]=np.array(precip.sel(Y=y[j],X=x[i],M=m))
            x1=regress(a,b_array1)
            error1[j,i,:]=np.square(b_array1-np.matmul(a,x1))
    b_array2=np.ones((tshape*4))
    for k,m in enumerate([1.0,2.0,3.0,4.0]):
        b_array2[k*tshape:(k+1)*tshape]=np.array(b2.sel(M=m))
    x2=regress(a,b_array2)
    error2=np.square(b_array2-np.matmul(a,x2))
    pc=np.empty(shape)
    for i in range(shape[1]):
        for j in range(shape[0]):
            pc[j,i]=np.corrcoef(error1[j,i,:],error2)[0,1]
    return pc
        
        


In [ ]:
windanom=wforcast_adjtime.groupby('T.month') - wforcast_adjtime.groupby('T.month').mean('T')
rainanom=pforcast_adjtime.groupby('T.month') - pforcast_adjtime.groupby('T.month').mean('T')
prep=rainanom.pr.sel(T=rainanom["T.month"]==10)
ind=windanom.ua.sel(T=windanom["T.month"]==10)
dep=sstforcast_adjtime.ts.sel(T=sstforcast_adjtime["T.month"]==10)
pc_10_wind_ind=partialcorr(ind,dep,prep,1,(28,33),79)

In [ ]:
plotmap(pc_10_wind_ind,"oj",resolution="one",ax=plt.axes(projection=ccrs.crs.Robinson()),colorbar="auto")

In [ ]:
month=["Nov","Dec","Jan","Feb"]
monthnum=[10,11,1,2]
fig, (ax1, ax2,ax3,ax4) = plt.subplots(ncols=4, subplot_kw={'projection': ccrs.crs.Robinson()},figsize=(30,5))
for i,j in enumerate([ax1,ax2,ax3,ax4]):
    pmonth=pforcast_adjtime.sel(T=pforcast_adjtime["T.month"]==monthnum[i])
    rainanom=pmonth - pmonth.mean('T')
    wmonth=wforcast_adjtime.sel(T=wforcast_adjtime["T.month"]==monthnum[i])
    sstmonth=sstforcast_adjtime.sel(T=wforcast_adjtime["T.month"]==monthnum[i])
    windanom=wmonth-wmonth.mean("T")
    plot=partialcorr(windanom.ua,sstmonth.ts,rainanom.pr,1,(28,33),np.shape(windanom.ua)[0])
    cb=plotmap(plot,"PC sst independent "+month[i],resolution="one",ax=j,colorbar="nonauto")
cbar=fig.add_axes([.9, 0.15, 0.02, 0.5])
fig.colorbar(cb,cax=cbar)

In [ ]:
month=["Nov","Dec","Jan","Feb"]
monthnum=[10,11,1,2]
fig, (ax1, ax2,ax3,ax4) = plt.subplots(ncols=4, subplot_kw={'projection': ccrs.crs.Robinson()},figsize=(30,5))
for i,j in enumerate([ax1,ax2,ax3,ax4]):
    pmonth=pforcast_adjtime.sel(T=pforcast_adjtime["T.month"]==monthnum[i])
    rainanom=pmonth - pmonth.mean('T')
    wmonth=wforcast_adjtime.sel(T=wforcast_adjtime["T.month"]==monthnum[i])
    sstmonth=sstforcast_adjtime.sel(T=wforcast_adjtime["T.month"]==monthnum[i])
    windanom=wmonth-wmonth.mean("T")
    plot=partialcorr(sstmonth.ts,windanom.ua,rainanom.pr,1,(28,33),np.shape(windanom.ua)[0])
    cb=plotmap(plot,"PC Wind independent "+month[i],resolution="one",ax=j,colorbar="nonauto")
cbar=fig.add_axes([.9, 0.15, 0.02, 0.5])
fig.colorbar(cb,cax=cbar)

Interpretation: For wind independent we are looking at the resulting correlation between sst and precipitation when their relationship with wind is removed.
Whereas when sst is indepeendent we are looking at the resulting correlation between wind and precipiation when sst is removed. 


Signifigance for sample size: 

In [ ]:
2/np.sqrt(77*4)

partialcorrcheck

In [ ]:
def makea(data,shape):
    a=np.ones((shape*4,2))
    for i,k in enumerate([1.0,2.0,3.0,4.0]):
        a[i*shape:i*shape+shape,0]=data.sel(M=k)
    return a 
def makeb(data,shape,isitprecip,y,x):
    if isitprecip==True:
        b=np.empty((shape*4))
        for i,k in enumerate([1.0,2.0,3.0,4.0]):
            b[i*shape:i*shape+shape]=data.sel(M=k,X=x,Y=y)
    else:
        b=np.empty((shape*4))
        for i,k in enumerate([1.0,2.0,3.0,4.0]):
            b[i*shape:i*shape+shape]=data.sel(M=k)
    return b 
def partialcorr2(a,b1,b2,x,y,tshape):
    b_2=makeb(b2,tshape,False,1,1)
    a=makea(a,tshape)
    error=np.empty((len(y),len(x)))
    x_2=regress(a,b_2)
    error2=np.square(b_2-np.matmul(a,x_2))
    for l,i in enumerate(y):
        for k,j in enumerate(x):
            b_1=makeb(b1,tshape,True,i,j)
            x_1=regress(a,b_1)
            error1=np.square(b_1-np.matmul(a,x_1))
            error[l,k]=np.cov(error1,error2)[0,1]/np.std(error1)/np.std(error2)
    return error 
x,y=getlonlat("one")

                
            
            

In [ ]:
np.shape(getlonlat("one")[0])
np.shape(getlonlat("one")[1])

In [ ]:
plotmap(partialcorr2(sstmonth.ts,rainanom.pr,windanom.ua,x,y,76),"oj",resolution="one",ax=plt.axes(projection=ccrs.crs.Robinson()),colorbar="auto")

In [ ]:
month=["Nov","Dec","Jan","Feb"]
monthnum=[10,11,1,2]
fig, (ax1, ax2,ax3,ax4) = plt.subplots(ncols=4, subplot_kw={'projection': ccrs.crs.Robinson()},figsize=(30,5))
for i,j in enumerate([ax1,ax2,ax3,ax4]):
    pmonth=pforcast_adjtime.sel(T=pforcast_adjtime["T.month"]==monthnum[i])
    rainanom=pmonth - pmonth.mean('T')
    wmonth=wforcast_adjtime.sel(T=wforcast_adjtime["T.month"]==monthnum[i])
    sstmonth=sstforcast_adjtime.sel(T=wforcast_adjtime["T.month"]==monthnum[i])
    windanom=wmonth-wmonth.mean("T")
    x,y=getlonlat("one")
    plot=partialcorr2(sstmonth.ts,rainanom.pr,windanom.ua,x,y,np.shape(windanom.ua)[0])
    cb=plotmap(plot,"PC sst independent "+month[i],resolution="one",ax=j,colorbar="nonauto")
cbar=fig.add_axes([.9, 0.15, 0.02, 0.5])
fig.colorbar(cb,cax=cbar)

In [ ]:
month=["Nov","Dec","Jan","Feb"]
monthnum=[10,11,1,2]
fig, (ax1, ax2,ax3,ax4) = plt.subplots(ncols=4, subplot_kw={'projection': ccrs.crs.Robinson()},figsize=(30,5))
for i,j in enumerate([ax1,ax2,ax3,ax4]):
    pmonth=pforcast_adjtime.sel(T=pforcast_adjtime["T.month"]==monthnum[i])
    rainanom=pmonth - pmonth.mean('T')
    wmonth=wforcast_adjtime.sel(T=wforcast_adjtime["T.month"]==monthnum[i])
    sstmonth=sstforcast_adjtime.sel(T=wforcast_adjtime["T.month"]==monthnum[i])
    windanom=wmonth-wmonth.mean("T")
    x,y=getlonlat("one")
    plot=partialcorr2(windanom.ua,rainanom.pr,sstmonth.ts,x,y,np.shape(windanom.ua)[0])
    cb=plotmap(plot,"PC Wind independent "+month[i],resolution="one",ax=j,colorbar="nonauto")
cbar=fig.add_axes([.9, 0.15, 0.02, 0.5])
fig.colorbar(cb,cax=cbar)